<a href="https://colab.research.google.com/github/shwets1503/Sales_Prediction/blob/master/Understanding_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# import libraries
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division

import plotly.graph_objects as go

In [15]:
customer = pd.read_csv('../content/drive/My Drive/sales_data/customers_dataset.csv', engine='python')
orders_df = pd.read_csv('../content/drive/My Drive/sales_data2/olist_orders_dataset.csv', engine='python')
order_items = pd.read_csv('../content/drive/My Drive/sales_data2/olist_order_items_dataset.csv', engine='python')

In [16]:
customer.shape

(99441, 5)

In [17]:
orders = pd.merge(customer, orders_df, how ='left', on='customer_id')

In [18]:
orders = pd.merge(orders, order_items,how ='left', on='order_id')

In [19]:
orders.shape

(113425, 18)

In [20]:
orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


In [21]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113425 entries, 0 to 113424
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    113425 non-null  object 
 1   customer_unique_id             113425 non-null  object 
 2   customer_zip_code_prefix       113425 non-null  int64  
 3   customer_city                  113425 non-null  object 
 4   customer_state                 113425 non-null  object 
 5   order_id                       113425 non-null  object 
 6   order_status                   113425 non-null  object 
 7   order_purchase_timestamp       113425 non-null  object 
 8   order_approved_at              113264 non-null  object 
 9   order_delivered_carrier_date   111457 non-null  object 
 10  order_delivered_customer_date  110196 non-null  object 
 11  order_estimated_delivery_date  113425 non-null  object 
 12  order_item_id                 

In [22]:
col = ['customer_unique_id', 'price', 'order_item_id', 'order_purchase_timestamp']
orders = orders[col]

Revenue = Active Customer Count * Order Count * Average Revenue per Order

In [23]:
#converting the type of Invoice Date Field from string to datetime.
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

#creating YearMonth field for the ease of reporting and visualization
orders['InvoiceYearMonth'] = orders['order_purchase_timestamp'].map(lambda date: 100*date.year + date.month)

#calculate Revenue for each row and create a new dataframe with YearMonth - Revenue columns
orders['Revenue'] = orders['price'] * orders['order_item_id']
orders_revenue = orders.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
orders.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp,InvoiceYearMonth,Revenue
0,861eff4711a542e4b93843c6dd7febb0,124.99,1.0,2017-05-16 15:05:35,201705,124.99
1,290c77bc529b7ac935b93aa66c333dc3,289.00,1.0,2018-01-12 20:48:24,201801,289.00
2,060e732b5b29e8181a18229c7b0b2b5e,139.94,1.0,2018-05-19 16:07:45,201805,139.94
3,259dac757896d24d7702b9acbbff3f3c,149.94,1.0,2018-03-13 16:06:38,201803,149.94
4,345ecd01c38d18a9036ed96c73b8d066,230.00,1.0,2018-07-29 09:51:30,201807,230.00


# Monthly Revenue

In [24]:
#X and Y axis inputs for Plotly graph. We use Scatter for line graphs
plot_data = [
    go.Scatter(
        x=orders_revenue['InvoiceYearMonth'],
        y=orders_revenue['Revenue'],
        mode = 'lines+markers'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Revenue', width=900, height=500
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

# Monthly Growth Rate

In [25]:
#using pct_change() function to see monthly percentage change
orders_revenue['MonthlyGrowth'] = orders_revenue['Revenue'].pct_change()

#showing first 5 rows
orders_revenue.head()

,InvoiceYearMonth,Revenue,MonthlyGrowth
0,201609,435.23,NaN
1,201610,56103.79,127.906073
2,201612,10.90,-0.999806
3,201701,142077.30,13033.614679
4,201702,269786.66,0.898872


In [26]:
#visualization - line graph
plot_data = [
    go.Scatter(
        x=orders_revenue.query("InvoiceYearMonth > 201701")['InvoiceYearMonth'],
        y=orders_revenue.query("InvoiceYearMonth > 201701")['MonthlyGrowth'],
        mode = 'lines+markers'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Growth Rate', width=900, height=500
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

# Monthly Active Customers

In [27]:
#creating a new dataframe with UK customers only
#tx_uk = tx_data.query("Country=='United Kingdom'").reset_index(drop=True)

#creating monthly active customers dataframe by counting unique Customer IDs
orders_monthly_active = orders.groupby('InvoiceYearMonth')['customer_unique_id'].nunique().reset_index()

#print the dataframe
orders_monthly_active.head()

,InvoiceYearMonth,customer_unique_id
0,201609,4
1,201610,321
2,201612,1
3,201701,765
4,201702,1755


In [28]:
#plotting the output
plot_data = [
    go.Bar(
        x=orders_monthly_active['InvoiceYearMonth'],
        y=orders_monthly_active['customer_unique_id'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Active Customers',width=900, height=500
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

# Monthly Total Number of Orders

In [29]:
#create a new dataframe for no. of order by using quantity field
orders_monthly_sales = orders.groupby('InvoiceYearMonth')['order_item_id'].sum().reset_index()

#print the dataframe
orders_monthly_sales.head()

,InvoiceYearMonth,order_item_id
0,201609,10.0
1,201610,457.0
2,201612,1.0
3,201701,1318.0
4,201702,2252.0


In [30]:
#plot
plot_data = [
    go.Bar(
        x=orders_monthly_sales['InvoiceYearMonth'],
        y=orders_monthly_sales['order_item_id'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order',width=900, height=500
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

# Monthly Orders Average

In [31]:
# create a new dataframe for average revenue by taking the mean of it
orders_monthly_order_avg = orders.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

#print the dataframe
orders_monthly_order_avg.head()

,InvoiceYearMonth,Revenue
0,201609,72.538333
1,201610,154.555895
2,201612,10.900000
3,201701,148.772042
4,201702,138.281220


In [32]:
#plot the bar chart
plot_data = [
    go.Bar(
        x=orders_monthly_order_avg['InvoiceYearMonth'],
        y=orders_monthly_order_avg['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Order Average',width=900, height=500
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

# New vs Existing Users

In [33]:
#create a dataframe contaning CustomerID and first purchase date
orders_min_purchase = orders.groupby('customer_unique_id').order_purchase_timestamp.min().reset_index()
orders_min_purchase.columns = ['customer_unique_id','MinPurchaseDate']
orders_min_purchase['MinPurchaseYearMonth'] = orders_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

#merge first purchase date column to our main dataframe (tx_uk)
orders = pd.merge(orders, orders_min_purchase, on='customer_unique_id')

In [34]:
#create a column called User Type and assign Existing 
#if User's First Purchase Year Month before the selected Invoice Year Month
orders['UserType'] = 'New'
orders.loc[orders['InvoiceYearMonth']>orders['MinPurchaseYearMonth'],'UserType'] = 'Existing'

#calculate the Revenue per month for each user type
orders_user_type_revenue = orders.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()
orders.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth,UserType
0,861eff4711a542e4b93843c6dd7febb0,124.99,1.0,2017-05-16 15:05:35,201705,124.99,2017-05-16 15:05:35,201705,New
1,290c77bc529b7ac935b93aa66c333dc3,289.00,1.0,2018-01-12 20:48:24,201801,289.00,2018-01-12 20:48:24,201801,New
2,060e732b5b29e8181a18229c7b0b2b5e,139.94,1.0,2018-05-19 16:07:45,201805,139.94,2018-05-19 16:07:45,201805,New
3,259dac757896d24d7702b9acbbff3f3c,149.94,1.0,2018-03-13 16:06:38,201803,149.94,2018-03-13 16:06:38,201803,New
4,345ecd01c38d18a9036ed96c73b8d066,230.00,1.0,2018-07-29 09:51:30,201807,230.00,2018-07-29 09:51:30,201807,New


In [35]:
orders.UserType.value_counts()

New         110963
Existing      2462
Name: UserType, dtype: int64

In [36]:
#filtering the dates and plot the result
orders_user_type_revenue = orders_user_type_revenue.query("InvoiceYearMonth != 201012 and InvoiceYearMonth != 201112")
plot_data = [
    go.Scatter(
        x=orders_user_type_revenue.query("UserType == 'Existing'")['InvoiceYearMonth'],
        y=orders_user_type_revenue.query("UserType == 'Existing'")['Revenue'],
        name = 'Existing'
    ),
    go.Scatter(
        x=orders_user_type_revenue.query("UserType == 'New'")['InvoiceYearMonth'],
        y=orders_user_type_revenue.query("UserType == 'New'")['Revenue'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New vs Existing'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

# New Customer Ratio

In [37]:
#create a dataframe that shows new user ratio - we also need to drop NA values (first month new user ratio is 0)
orders_user_ratio = orders.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['customer_unique_id'].nunique()/orders.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['customer_unique_id'].nunique() 
orders_user_ratio = orders_user_ratio.reset_index()
orders_user_ratio = orders_user_ratio.dropna()

#print the dafaframe
orders_user_ratio

#plot the result

plot_data = [
    go.Bar(
        x=orders_user_ratio.query("InvoiceYearMonth>201701 and InvoiceYearMonth<201806")['InvoiceYearMonth'],
        y=orders_user_ratio.query("InvoiceYearMonth>201701 and InvoiceYearMonth<201806")['customer_unique_id'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New Customer Ratio'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

# Monthly Retention rate

In [38]:
#identify which users are active by looking at their revenue per month
orders_user_purchase = orders.groupby(['customer_unique_id','InvoiceYearMonth'])['Revenue'].sum().reset_index()

#create retention matrix with crosstab
orders_retention = pd.crosstab(orders_user_purchase['customer_unique_id'], orders_user_purchase['InvoiceYearMonth']).reset_index()

orders_retention.head()

InvoiceYearMonth,customer_unique_id,201609,201610,201612,201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810
0,0000366f3b9a7992bf8c76cfdf3221e2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0000f46a3911fa3c0805444483337064,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0004aac84e0df4da2b147fca70cf8255,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [39]:
#create an array of dictionary which keeps Retained & Total User count for each month
months = orders_retention.columns[2:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['InvoiceYearMonth'] = int(selected_month)
    retention_data['TotalUserCount'] = orders_retention[selected_month].sum()
    retention_data['RetainedUserCount'] = orders_retention[(orders_retention[selected_month]>0) & (orders_retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)
    
#convert the array to dataframe and calculate Retention Rate
orders_retention = pd.DataFrame(retention_array)
orders_retention['RetentionRate'] = orders_retention['RetainedUserCount']/orders_retention['TotalUserCount']

In [40]:
#plot the retention rate graph
plot_data = [
    go.Scatter(
        x=orders_retention.query("InvoiceYearMonth<201806")['InvoiceYearMonth'],
        y=orders_retention.query("InvoiceYearMonth<201806")['RetentionRate'],
        name="organic"
    )
    
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Retention Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

In [41]:
#create our retention table again with crosstab() and add firs purchase year month view
orders_retention = pd.crosstab(orders_user_purchase['customer_unique_id'], orders_user_purchase['InvoiceYearMonth']).reset_index()
orders_retention = pd.merge(orders_retention,orders_min_purchase[['customer_unique_id','MinPurchaseYearMonth']],on='customer_unique_id')
new_column_names = [ 'm_' + str(column) for column in orders_retention.columns[:-1]]
new_column_names.append('MinPurchaseYearMonth')
orders_retention.columns = new_column_names

#create the array of Retained users for each cohort monthly
retention_array = []
for i in range(len(months)):
    retention_data = {}
    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i+1:]
    for prev_month in prev_months:
        retention_data[prev_month] = np.nan
        
    total_user_count = orders_retention[orders_retention.MinPurchaseYearMonth ==  selected_month].MinPurchaseYearMonth.count()
    retention_data['TotalUserCount'] = total_user_count
    retention_data[selected_month] = 1 
    
    query = "MinPurchaseYearMonth == {}".format(selected_month)
    

    for next_month in next_months:
        new_query = query + " and {} > 0".format(str('m_' + str(next_month)))
        retention_data[next_month] = np.round(orders_retention.query(new_query)['m_' + str(next_month)].sum()/total_user_count,2)
    retention_array.append(retention_data)
    
orders_retention = pd.DataFrame(retention_array)
orders_retention.index = months

#showing new cohort based retention table
orders_retention.head()

,TotalUserCount,201610,201612,201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810
InvoiceYearMonth,,,,,,,,,,,,,,,,,,,,,,,,,
201610,321,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0
201612,1,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
201701,764,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
201702,1752,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
201703,2636,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
